#CRC Chemotherapy Response Analysis Notebook

This notebook outlines the main steps for single-cell RNA sequencing (scRNA-seq)
data analysis of Colorectal Cancer (CRC) patients undergoing chemotherapy,
focusing on comparing responders versus progressors.

This version assumes you have already loaded a single, concatenated AnnData object
with the following columns in `adata.obs`:
- 'sample': The sample or patient ID.
- 'sample2': The treatment response type, containing 'responders' and 'progressors'.

The workflow covers:
1.  Data Loading (single concatenated AnnData object)
2.  Quality Control and Pre-processing
3.  Data Integration (assuming a batch-corrected latent space like `X_scVI` exists)
4.  Cell Type Annotation
5.  Tumor Microenvironment (TME) Analysis
6.  Tumor Cell Detection based on Copy Number Variations (CNV)
7.  Differential Gene Expression (DEG) Analysis with Downsampling
8.  Molecular Pathway (MP) Signatures Analysis (GSEA)


###1. Setup and Library Imports

In [ ]:
Install necessary packages if not already installed
# !pip install anndata hdf5plugin scanpy scvi-tools decoupler infercnvpy wget gseapy

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import anndata
import hdf5plugin
import scanpy as sc
# import scvi # Uncomment if you are running scVI directly within this notebook
import decoupler as dc
import infercnvpy as cnv
import wget # For downloading gencode annotation
from functools import cache # For caching gencode gene info parsing
import math
from multiprocessing import Pool, cpu_count

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib parameters for consistent plotting
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
sns.reset_defaults() # Reset seaborn defaults for fresh plots

# Mount Google Drive (if running in Google Colab)
# from google.colab import drive
# drive.mount('/content/drive')

###2. Data Loading (Single Concatenated AnnData object)bold text

In [ ]:
--- Load your pre-processed, concatenated AnnData object here ---
# The code in the `untitled_1.py` suggests loading multiple AnnData objects
# and concatenating them. An example is provided below.
# This assumes the AnnData objects contain necessary columns like 'sample' and 'sample2'.

In [ ]:
try:
    # Example from `untitled_1.py`:
    # pre = anndata.read_h5ad('/mnt/beegfs/userdata/g_puzanov/pre_TC')
    # MSI = anndata.read_h5ad('MSI_CRC')
    # adata = anndata.read_h5ad('/mnt/beegfs/userdata/g_puzanov/MSI+MSS')

    # Placeholder for a single concatenated file as per the original notebook plan.
    # Replace this path with the path to your AnnData file
    adata = sc.read_h5ad('path/to/your/single_concatenated_adata.h5ad')
    print(f"Loaded concatenated AnnData object with {adata.n_obs} cells and {adata.n_vars} genes.")

    # Check for the required columns
    if 'sample' not in adata.obs.columns or 'sample2' not in adata.obs.columns:
        raise ValueError("AnnData object does not contain 'sample' and 'sample2' columns.")

    print("Found 'sample' and 'sample2' columns in adata.obs.")

except FileNotFoundError:
    print("Please update the data loading path to your actual .h5ad file.")
    print("Example: adata = sc.read_h5ad('path/to/your/adata_file.h5ad')")
    # Exit or handle error gracefully if data isn't found
    # For now, we'll create a dummy object to allow the code to run
    print("Creating a dummy AnnData object for demonstration.")
    adata = sc.AnnData(np.random.rand(100, 50))
    adata.obs['sample'] = [f's{i//10}' for i in range(100)]
    adata.obs['sample2'] = ['responders'] * 50 + ['progressors'] * 50
    adata.var_names = [f'gene{i}' for i in range(50)]
    adata.var['mt'] = False
    adata.var['ribo'] = False
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    adata.raw = adata

###3. Clinical Data Integration

In [ ]:
print("\n--- Integrating Clinical Data ---")
# and adding it to the AnnData object. This section demonstrates how to do that.

# Assume clinical data files are available.
# cl = pd.read_excel('Samples_summary.xlsx')
# R = cl[cl['response'].isin(['PR','CR','SD'])]
# N = cl[cl['response'].isin(['PD'])]

# Example of adding `KRAS_status` based on clinical data
# The code from `untitled_1.py` first loads and cleans clinical data, then
# iterates through samples to assign KRAS status.

# Here is an example of the kind of clinical data processing and integration
# that was present in the provided file. This is crucial for downstream analysis.
if 'RAS/RAF mutation' in adata.obs.columns:
    adata.obs['KRAS_status'] = adata.obs['RAS/RAF mutation']
    for i in adata.obs['RAS/RAF mutation'].unique().tolist():
        if i in ['KRAS G12A','KRAS G12C','KRAS c.34G>T', 'KRAS c.34G>C','KRAS c.38G>A','KRAS (p.Gly12Val)', 'KRAS c.35G>A', 'KRAS c.59C>T','KRAS c.436C>G', 'KRAS A146T, ']:
            adata.obs['KRAS_status'] = adata.obs['KRAS_status'].replace(i, 'KRAS_mut')
        if (i in ['Not applicable']) or (isinstance(i, float) and math.isnan(i)):
            adata.obs['KRAS_status'] = adata.obs['KRAS_status'].replace(i, 'KRAS_WT')
    # Manually correct specific samples, as seen in `untitled_1.py`
    adata.obs.loc[adata.obs['sample'] == '23', 'KRAS_status'] = 'KRAS_mut'
    adata = adata[adata.obs['KRAS_status'].isin(['KRAS_mut','KRAS_WT'])]
    print("Integrated KRAS status from clinical data.")


# Example of adding `Site of metastasis`
if 'Site of Biopsy' in adata.obs.columns:
    adata.obs['Site of metastasis'] = adata.obs['sample']
    for i in adata.obs['sample'].unique().tolist():
        if adata[adata.obs['sample'] == i].obs['Site of Biopsy'].unique().tolist()[0] in ['Liver', 'Liver ']:
            adata.obs['Site of metastasis'] = adata.obs['Site of metastasis'].replace(i, 'Liver')
        else:
            adata.obs['Site of metastasis'] = adata.obs['Site of metastasis'].replace(i, 'Other')
    print("Integrated Site of metastasis from clinical data.")

### 4. Quality Control and Pre-processing (if not already done)

In [ ]:
# This section is included in case the loaded data needs a final QC pass.
# If your data is already fully QC'd, you can skip this block.

print("\n--- Performing a final QC and pre-processing step ---")

# Make variable names unique if they aren't already
adata.var_names_make_unique()

# Calculate QC metrics
adata.var['mt'] = adata.var_names.str.startswith(('mt-', 'MT-'))
adata.var['ribo'] = adata.var_names.str.startswith(('Rps', 'Rpl', 'RPS', 'RPL'))
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt', 'ribo'], inplace=True, percent_top=None, log1p=False)

# Filter cells based on QC metrics
min_genes_per_cell = 200
max_pct_mt = 20

initial_cells = adata.n_obs
adata = adata[
    (adata.obs.n_genes_by_counts > min_genes_per_cell) &
    (adata.obs.pct_counts_mt < max_pct_mt), :
]
print(f"Filtered {initial_cells - adata.n_obs} cells based on QC metrics.")

# Filter genes expressed in very few cells
min_cells_per_gene = 3
initial_genes = adata.n_vars
sc.pp.filter_genes(adata, min_cells=min_cells_per_gene)
print(f"Filtered {initial_genes - adata.n_vars} genes expressed in < {min_cells_per_gene} cells.")

# Normalize and log-transform if not already done
# The code below will be skipped if `adata.raw` exists, assuming raw counts were stored.
if adata.raw is None:
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    adata.raw = adata # Store the raw counts
    print("Normalized and log-transformed data. Raw counts stored.")

###5. Data Integration and Dimensionality Reduction (scVI)

In [ ]:
print("\n--- Data Integration and Dimensionality Reduction ---")

# This section assumes scVI has been run externally and corrected data (e.g., X_scVI)
# is either loaded or will be generated here. If you are loading an already
# scVI-corrected .h5ad file, you can skip the scvi.model.SCVI part.

# If your data is already corrected and loaded, use the latent representation
if 'X_scVI' not in adata.obsm:
    print("Warning: 'X_scVI' not found in adata.obsm. Proceeding with PCA for neighbors/UMAP.")
    sc.tl.pca(adata, svd_solver='arpack') # Fallback to PCA
    use_representation = 'X_pca'
else:
    print("Using scVI latent representation for downstream analysis.")
    use_representation = 'X_scVI'

# Compute neighbors and UMAP for visualization and clustering
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=50, use_rep=use_representation)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5) # Adjust resolution as needed for desired cluster granularity
print("Computed neighbors, UMAP, and Leiden clustering.")

# Visualize UMAP (optional)
# sc.pl.umap(adata, color=['leiden', 'sample', 'sample2'], title="UMAP of Integrated Data")
# plt.show()


###6. Cell Type Annotation

In [ ]:
# Download PanglaoDB markers (if not already done)
gencode_gff3_path = 'gencode.v44.annotation.gff3.gz'
if not os.path.exists(gencode_gff3_path):
    print(f"Downloading Gencode annotation to {gencode_gff3_path}...")
    wget.download('https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.annotation.gff3.gz', out=gencode_gff3_path)
    print("Download complete.")

# For demonstration, we'll use a simplified marker set or assume it's pre-loaded
markers = dc.get_resource('PanglaoDB') # Decoupler can directly fetch some resources

# Filter markers for human and remove duplicates
markers = markers[markers['human'] == 'True']
markers = markers[~markers.duplicated(['cell_type', 'genesymbol'])]

# Define immune cell types for CNV reference (adjust based on your data)
immune_cell_types = [
    'B cells', 'Macrophages', 'T cells', 'NK cells', 'Dendritic cells', 'Monocytes',
    'Neutrophils', 'Plasma cells', 'Mast cells', 'Eosinophils', 'Basophils'
]
markers = markers[markers['cell_type'].isin(immune_cell_types + ['Epithelial cells', 'Fibroblasts', 'Endothelial cells'])]


# Run Over-Representation Analysis (ORA) to assign cell types to Leiden clusters
dc.run_ora(mat=adata, net=markers, source='cell_type', target='genesymbol', min_n=5, verbose=True) # min_n: min genes for enrichment
acts = dc.get_acts(adata, obsm_key='ora_estimate')
mean_enr = dc.summarize_acts(acts, groupby='leiden', min_std=0.5) # Summarize enrichment per Leiden cluster

# Assign cell types to clusters based on highest enrichment
annotation_dict = dc.assign_groups(mean_enr)
adata.obs['cell_type_annotated'] = [annotation_dict[clust] for clust in adata.obs['leiden']]
print("Cell types annotated based on marker gene enrichment.")

# Re-labeling cell types as seen in `untitled_1.py`
# This is useful for grouping similar cell types or simplifying for CNV reference.
adata.obs['celltype'] = adata.obs['cell_type_annotated'].copy()
adata.obs['celltype'] = adata.obs['celltype'].replace('B cells/B naive', 'Non-tumor cells')
adata.obs['celltype'] = adata.obs['celltype'].replace('DC', 'Non-tumor cells')
# ... and so on for all non-tumor cell types

# Visualize UMAP with annotated cell types (optional)
# sc.pl.umap(adata, color='cell_type_annotated', legend_loc='on data', title="UMAP with Annotated Cell Types")
# plt.show()


###7. Tumor Cell Detection based on CNV (using infercnvpy)

In [ ]:
 Parse Gencode GFF3 for gene locations
@cache
def parse_gencode_gff3(file_path):
    gencode = pd.read_csv(file_path, comment="#", sep="\t",
                          names=['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'],
                          compression='gzip')
    gencode_genes = gencode[(gencode.feature == "gene")][['seqname', 'start', 'end', 'attribute']].copy().reset_index().drop('index', axis=1)

    def extract_gene_info(attr_str):
        g_name = next((x.split("=")[1] for x in attr_str.split(";") if 'gene_name' in x), None)
        g_type = next((x.split("=")[1] for x in attr_str.split(";") if 'gene_type' in x), None)
        return g_name, g_type

    gencode_genes["gene_name"], gencode_genes["gene_type"] = zip(*gencode_genes.attribute.apply(lambda x: extract_gene_info(x)))
    return gencode_genes[['seqname', 'start', 'end', 'gene_name', 'gene_type']].set_index('gene_name')

gencode_genes_df = parse_gencode_gff3(gencode_gff3_path)

# Add chromosome and position information to adata.var
print("Annotating gene locations for CNV analysis...")
adata.var['chromosome'] = adata.var_names.map(gencode_genes_df['seqname'])
adata.var['start'] = adata.var_names.map(gencode_genes_df['start'])
adata.var['end'] = adata.var_names.map(gencode_genes_df['end'])

# Filter out genes without genomic coordinates
adata = adata[:, adata.var['chromosome'].notna()]
print(f"Filtered to {adata.n_vars} genes with genomic coordinates.")

# Define reference cell types for CNV inference (non-malignant cells)
immune_cell_types_for_cnv = ['B cells/B naive', 'DC', 'Mono/Macro', 'NK', 'Plasma cells', 'pDC', 'CD8Tem', 'Th1', 'Tfh', 'Treg', 'CD8Teff']
reference_categories = [ct for ct in adata.obs['celltype'].unique() if ct in immune_cell_types_for_cnv]
print(f"Reference cell types for CNV: {reference_categories}")

# Run infercnv
try:
    cnv.tl.infercnv(
        adata,
        reference_key="celltype",
        reference_cat=reference_categories,
        window_size=250, # Size of the genomic window
        exclude_chromosomes=['chrX', 'chrY', 'chrM'],
    )
    print("infercnv analysis complete.")

    cnv.tl.pca(adata)
    cnv.pp.neighbors(adata)
    cnv.tl.leiden(adata, resolution=0.5, key_added='cnv_leiden')
    cnv.tl.umap(adata, key_added='cnv_umap')
    cnv.tl.cnv_score(adata)

    print("CNV PCA, neighbors, UMAP, Leiden, and score calculated.")

    # Plot heatmap (as seen in `untitled_1.py`)
    # cnv.pl.chromosome_heatmap(adata, groupby="celltype", save='CNV_Post.png')
    # cnv.pl.chromosome_heatmap(adata[adata.obs["celltype"] != "Non-tumor cells", :], groupby="celltype", dendrogram = True, save='CNV_Post_T.png')


    # Assign tumor/normal status based on CNV Leiden clusters
    # **UPDATE THIS BASED ON YOUR CNV ANALYSIS**
    # The snippet from `untitled_1.py` is based on a specific `adataT` object.
    # tumor_cnv_clusters = ['1', '2', '5']
    tumor_cnv_clusters = ['0', '1', '2']

    adata.obs["cnv_status"] = "normal"
    adata.obs.loc[adata.obs["cnv_leiden"].isin(tumor_cnv_clusters), "cnv_status"] = "tumor"
    print(f"Assigned {np.sum(adata.obs['cnv_status'] == 'tumor')} cells as 'tumor' based on CNV.")

    # Update main cell type column to reflect tumor status
    adata.obs['final_cell_type'] = adata.obs['celltype'].copy()
    adata.obs.loc[adata.obs['cnv_status'] == 'tumor', 'final_cell_type'] = 'Tumor_Malignant'
    print("Final cell type assignment complete, including tumor status.")

except Exception as e:
    print(f"infercnvpy analysis failed: {e}")
    print("Skipping CNV-based tumor cell detection and falling back to marker-based annotation.")
    adata.obs['final_cell_type'] = adata.obs['celltype'].copy()

###8. Differential Gene Expression (DEG) Analysis with Downsampling

In [ ]:
target_cell_type = 'Tumor_Malignant'
comparison_groups = ['responders', 'progressors']

adata_subset = adata[
    (adata.obs['final_cell_type'] == target_cell_type) &
    (adata.obs['sample2'].isin(comparison_groups))
].copy()

if adata_subset.n_obs == 0:
    print(f"No cells found for {target_cell_type} in {comparison_groups} for DEG analysis.")
else:
    print(f"Subset for DEG analysis: {adata_subset.n_obs} cells of type '{target_cell_type}'.")

    # Downsampling for balanced comparison
    min_cells_per_group = min(adata_subset.obs['sample2'].value_counts())
    if min_cells_per_group == 0:
        print("Cannot downsample: one of the groups has 0 cells.")
        adata_downsampled = adata_subset
    else:
        print(f"Downsampling each group to {min_cells_per_group} cells.")
        adata_downsampled_list = []
        for group in comparison_groups:
            group_adata = adata_subset[adata_subset.obs['sample2'] == group].copy()
            sampled_indices = np.random.choice(group_adata.obs_names, min_cells_per_group, replace=False)
            adata_downsampled_list.append(group_adata[sampled_indices].copy())
        adata_downsampled = sc.concat(adata_downsampled_list, axis=0)
        print(f"Downsampled data: {adata_downsampled.n_obs} cells.")

    # Perform DEG analysis
    sc.tl.rank_genes_groups(adata_downsampled, groupby='sample2', reference='responders', method='wilcoxon', key_added='deg_results')
    print("DEG analysis complete.")

    # Extract and save DEGs
    degs_df = sc.get.rank_genes_groups_df(adata_downsampled, group='progressors', key='deg_results')
    degs_df_filtered = degs_df[(degs_df['pvals_adj'] < 0.05) & (abs(degs_df['logfoldchanges']) > 0.25)]
    print(f"Found {len(degs_df_filtered)} significant DEGs (adj. p < 0.05, |logFC| > 0.25) in Progressors vs Responders.")

    output_deg_path = 'drive/MyDrive/DEGs_Progressors_vs_Responders_TumorMalignant.csv'
    degs_df_filtered.to_csv(output_deg_path)
    print(f"Filtered DEGs saved to: {output_deg_path}")

###9. Molecular Pathway (MP) Signatures Analysis (GSEA)

In [ ]:
# Here's an example of how you might use `decoupler` for this task.
# This assumes `acts` was calculated in the cell type annotation step.
if 'ora_estimate' in adata.obsm:
    df = dc.rank_sources_groups(acts, groupby='sample2', reference='non-response', method='wilcoxon')
    df = df[df['pvals_adj'] < 0.05]
    print(f"Found {len(df)} significant pathways.")
    # dc.plot_barplot(df, 'meanchange', top=25, vertical=True, save = 'Enrich_CSC_NR.png')
else:
    print("ORA estimates not found in `adata.obsm`. Skipping pathway analysis.")

###10. Final Data Saving

In [ ]:
# Ensure column types are compatible for .h5ad saving
def sanitize_types_for_h5(adata_obj):
    """Converts object columns in .obs and .var to numeric if possible,
    or fills with NaN for compatibility with h5ad saving."""
    for df in [adata_obj.obs, adata_obj.var]:
        for col in df.columns:
            if df[col].dtype == object:
                numeric_col = pd.to_numeric(df[col], errors='coerce')
                if not numeric_col.isna().all():
                    df[col] = numeric_col
                else:
                    df[col] = df[col].astype(str).replace('None', np.nan)
            df[col] = df[col].astype(str)

    if adata_obj.raw is not None:
        for col in adata_obj.raw.var.columns:
            if adata_obj.raw.var[col].dtype == object:
                numeric_col = pd.to_numeric(adata_obj.raw.var[col], errors='coerce')
                if not numeric_col.isna().all():
                    adata_obj.raw.var[col] = numeric_col
                else:
                    adata_obj.raw.var[col] = adata_obj.raw.var[col].astype(str).replace('None', np.nan)
            adata_obj.raw.var[col] = adata_obj.raw.var[col].astype(str)

    adata_obj.obs.columns = adata_obj.obs.columns.astype(str)
    adata_obj.var.columns = adata_obj.var.columns.astype(str)
    if adata_obj.raw is not None:
        adata_obj.raw.var.columns = adata_obj.raw.var.columns.astype(str)

    return adata_obj

print("\n--- Saving Final AnnData Object ---")
adata = sanitize_types_for_h5(adata)

output_adata_path = 'drive/MyDrive/CRC_Chemotherapy_Analysis_Final.h5ad'
adata.write_h5ad(
    output_adata_path,
    compression=hdf5plugin.FILTERS["zstd"]
)
print(f"Final AnnData object saved to: {output_adata_path}")

print("\nAnalysis workflow complete. Remember to review and interpret your results!")